<a href="https://colab.research.google.com/github/ZhiyuanH3/BadmintonDataAnalysis/blob/master/BWF_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Import all libraries

#import requests
#import urllib.request 
from time import sleep as slp
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen

import re
import pandas as pd
import numpy as np

# URL of the target website
root = 'https://bwf.tournamentsoftware.com/sport/'
url = root+'player.aspx?id=D3303F12-E5BF-4D78-BB58-9E1236DD64A1&player=489'

uClient = urlopen(url)

#out_name = './out.csv'
#headers = 'BWF Kento Momota'

#f = open(out_name, 'w')
#f.write(headers)

In [0]:
# Get the elements of the page and close the current client
soup1 = soup(uClient.read(), 'lxml') #'html.parser')
uClient.close()

In [0]:
#print(soup1.prettify())

In [0]:
# Find the html div tags with class='tabbedtournamentlist', which stores the tournament list 
containers = soup1.find_all('div', {'class': 'tabbedtournamentlist'})

In [0]:
len(containers)

1

In [0]:
# These lists store the tournament info
#tournaments = []
hrefs = []
dates = []

for container in containers[0].find_all('table'):
  trs = container.find_all('tr')
  for tr in trs:
    #print('_________________')
    a = tr.find('a')
    if a != None:
      hrefs.append(a['href'])
      #print(a['href'])

    td = tr.find('td')
    if td != None and td['class'][0]=='dates':
      dates.append(td.find_all(text=True)[0])  
    
#print(len(tournaments))

In [0]:
# The amount of BWF tournaments the player has participated in, which were registrated  
len(dates)

147

In [0]:
tournament_info = []
tournament_dfs = []

cc = 0

for href_i in hrefs:


  Tour = []
  Time = []
  Player1 = []
  Player2 = []
  Score = []
  Stats_href = []

  Duration = []
  Place = []

  url_i = root + href_i 

  uClient_i = urlopen(url_i)
  soup_i = soup(uClient_i.read(), 'lxml')
  uClient_i.close()

  containers_i = soup_i.find_all('tbody')


  # The following commented codes won't work, since some of the events lack scores
  """
  for pt in containers_i[0].find_all('td', {'class': 'plannedtime'}):
    print(pt.text)
    Time.append(pt.text)

  for plr in containers_i[0].find_all('a', href=re.compile('player')):
    print(plr.text)
    if len(Player1) == len(Player2):
      Player1.append(plr.text)
    else:
      Player2.append(plr.text)  

  #for sc in containers_i[0].find_all('span', {'class': 'score'}):
  #  print(sc.text)
  #  Score.append(sc.text)
  """

  # Get the link for each event for further info
  for stats_href in containers_i[0].find_all('a', {'class': 'icon stats'}):
    print(stats_href)
    Stats_href.append(stats_href['href']) 



  # Loop over all matches for each tournament
  for href_j in Stats_href:
    # The complete url
    url_j = root + href_j

    uClient_j = urlopen(url_j)
    soup_j = soup(uClient_j.read(), 'lxml')
    uClient_j.close()


    # Get the name of the tournament
    tour = '?'
    tour = soup_j.find('div', {'class': 'media__content'}).find('span', {'class': 'nav-link__value'}).text
    print(tour)
    Tour.append(tour)


    # Get the place in which the tournament takes place
    place = '?'
    nlvs = soup_j.find('div', {'class': 'media__content'}).find_all('span', {'class': 'nav-link__value'})
    place = nlvs[2].text
    print(place)
    Place.append(place)


    # Get the time when of the match takes place
    time = soup_j.find('td', {'class': 'plannedtime'})
    # We filled the time with ? for those matches that has no registrated time
    if time:
      print(time.text)
      Time.append(time.text)    
    else:
      Time.append('?')


    # Get the names of the players
    p1 = soup_j.find('a', {'id': 'lnk1'})
    p2 = soup_j.find('a', {'id': 'lnk3'})
    Player1.append(p1.text)
    Player2.append(p2.text)


    # Get the duration of the match
    ll = len(Duration)
    containers_j = soup_j.find_all('td', {'align': 'right'})
  
    for container_j in containers_j:
      print(container_j)
      # The format of the duration is 'hour:minute'
      duration = container_j.find(text=re.compile(':'))
      #duration = container_j.find_all(text=re.compile('\d+:{1}\d+(?!\s+PM|\s+AM)'))
      if duration:
        if 'M' in duration:
          continue
        else:  
          print(duration)
          Duration.append(duration)
          break
    # In a waklover case, there are no duration registrated. Let's fill in a '?'
    # to keep the length of the list consistent
    if len(Duration) == ll:
      Duration.append('?')  
    

    # Get the score of the match
    containers_j = soup_j.find_all('span', {'class': 'score'})
    
    for container_j in containers_j:
      # The format of the score line is 'score_player1-score_player2'
      score_j = container_j.find_all(text=re.compile('-'))
      if not score_j:
        # In case of a walkover, the corresponding string is then 'Walkover'
        score_j = container_j.find_all(text=re.compile('alkover')) 
      if score_j:
        print(score_j)
        Score.append(score_j)
        break

    

  print(Score)
  print(Duration)
  # Build a DataFrame for each tournament
  df_i = pd.DataFrame()
  df_i['Tournament'] = Tour
  df_i['Time'] = Time
  df_i['Place'] = Place
  df_i['Player1'] = Player1
  df_i['Player2'] = Player2
  df_i['Score'] = Score
  df_i['Duration'] = Duration
  
  df_i['Stats_href'] = Stats_href
  

  # Each DataFrame are element of the following list, which is going to be concatenated into the final table
  tournament_info.append([Tour, Time, Place, Player1, Player2, Score, Duration, Stats_href]) 
  tournament_dfs.append(df_i)


  # For debugging
  cc += 1
  #if cc == 2:  break

[]
[]
<a class="icon stats" href="./match.aspx?id=6D74F318-A951-4C8F-9451-5DA3F33DF6C3&amp;match=44"><img class="intext" height="14" src="//static.tournamentsoftware.com/images/icon_stats.gif" title="View match statistics" width="18"/></a>
<a class="icon stats" href="./match.aspx?id=6D74F318-A951-4C8F-9451-5DA3F33DF6C3&amp;match=36"><img class="intext" height="14" src="//static.tournamentsoftware.com/images/icon_stats.gif" title="View match statistics" width="18"/></a>
<a class="icon stats" href="./match.aspx?id=6D74F318-A951-4C8F-9451-5DA3F33DF6C3&amp;match=32"><img class="intext" height="14" src="//static.tournamentsoftware.com/images/icon_stats.gif" title="View match statistics" width="18"/></a>
PERODUA Malaysia Masters 2020
Badminton World Federation | Kuala Lumpur, Malaysia
Wed 1/8/2020
<td align="right">Duration</td>
<td align="right" class="plannedtime">Wed 1/8/2020</td>
<td align="right"><table align="Right">
<tr>
<td align="right"><strong><a href="player.aspx?id=6D74F318-A951-

In [0]:
df_out = pd.concat(tournament_dfs, axis=0)

In [0]:
df_out.head(23)

,Tournament,Time,Place,Player1,Player2,Score,Duration,Stats_href
0,PERODUA Malaysia Masters 2020,Wed 1/8/2020,"Badminton World Federation | Kuala Lumpur, Mal...",[JPN] Kento MOMOTA,Kashyap PARUPALLI[IND],"[21-17, 21-16]",0:43,./match.aspx?id=6D74F318-A951-4C8F-9451-5DA3F3...
1,PERODUA Malaysia Masters 2020,Thu 1/9/2020,"Badminton World Federation | Kuala Lumpur, Mal...",[JPN] Kento MOMOTA,H. S. PRANNOY[IND],"[21-14, 21-16]",0:45,./match.aspx?id=6D74F318-A951-4C8F-9451-5DA3F3...
2,PERODUA Malaysia Masters 2020,Fri 1/10/2020,"Badminton World Federation | Kuala Lumpur, Mal...",[JPN] Kento MOMOTA,Yu Xiang HUANG[CHN],"[21-13, 19-21, 21-9]",1:07,./match.aspx?id=6D74F318-A951-4C8F-9451-5DA3F3...
0,HSBC BWF World Tour Finals 2019,Wed 12/11/2019,"Badminton World Federation | Guangzhou, China",[JPN] Kento Momota,Tzu Wei Wang[TPE],"[21-10, 21-9]",0:29,./match.aspx?id=EF144033-A23C-47A2-9CC7-4F2922...
1,HSBC BWF World Tour Finals 2019,Thu 12/12/2019,"Badminton World Federation | Guangzhou, China",[JPN] Kento Momota,Anders Antonsen[DEN],"[19-21, 21-15, 21-15]",1:23,./match.aspx?id=EF144033-A23C-47A2-9CC7-4F2922...
2,HSBC BWF World Tour Finals 2019,Fri 12/13/2019,"Badminton World Federation | Guangzhou, China",[JPN] Kento Momota,Jonatan Christie[INA],"[21-14, 21-14]",0:38,./match.aspx?id=EF144033-A23C-47A2-9CC7-4F2922...
3,HSBC BWF World Tour Finals 2019,Sat 12/14/2019,"Badminton World Federation | Guangzhou, China",[JPN] Kento Momota,Tzu Wei Wang[TPE],"[21-17, 21-12]",0:48,./match.aspx?id=EF144033-A23C-47A2-9CC7-4F2922...
4,HSBC BWF World Tour Finals 2019,Sun 12/15/2019,"Badminton World Federation | Guangzhou, China",[JPN] Kento Momota,Anthony Sinisuka Ginting[INA],"[17-21, 21-17, 21-14]",1:27,./match.aspx?id=EF144033-A23C-47A2-9CC7-4F2922...
0,Fuzhou China Open 2019,Tue 11/5/2019,"Badminton World Federation | Fuzhou, China",[JPN] Kento Momota,Daren Liew[MAS],[21-17],0:22,./match.aspx?id=D3303F12-E5BF-4D78-BB58-9E1236...
1,Fuzhou China Open 2019,Thu 11/7/2019,"Badminton World Federation | Fuzhou, China",[JPN] Kento Momota,Zii Jia Lee[MAS],"[21-9, 11-3]",0:21,./match.aspx?id=D3303F12-E5BF-4D78-BB58-9E1236...


In [0]:
df_out.to_csv('Momota_dataset.csv')